In [1]:
import os
import json
import numpy as np
from PIL import Image


In [54]:
data_root = '/data_new/chence/CelebA/Img/img_celeba.7z'
data_source = 'CelebA'
print('data_root', data_root)
print('data_source', data_source)


data_root /data_new/chence/CelebA/Img/img_celeba.7z
data_source CelebA


In [55]:
align_images_dir = os.path.join(data_root, 'align_images')
align_parsing_dir = os.path.join(data_root, 'align_parsing')
head_images_dir = os.path.join(data_root, 'head_images')
head_parsing_dir = os.path.join(data_root, 'head_parsing')
assert os.path.exists(align_images_dir)
assert os.path.exists(align_parsing_dir)
assert os.path.exists(head_images_dir)
assert os.path.exists(head_parsing_dir)

meta_files = [i for i in os.listdir(data_root) if i.endswith('.json') and i.startswith('meta')]
meta_files = sorted(meta_files, key=lambda x: int(x.split('_')[1].split('.')[0].split('-')[0]))
print(meta_files)

['meta_1-3.json', 'meta_2-3.json', 'meta_3-3.json']


In [56]:
def get_cam_coords(c2w):
    # World Coordinate System: x(right), y(up), z(forward)
    T = c2w[:3, 3]
    x, y, z = T
    r = np.sqrt(x**2+y**2+z**2)
    # theta = np.rad2deg(np.arctan(np.sqrt(x**2+z**2)/y))
    theta = np.rad2deg(np.arctan2(x, z))
    if theta >= -90 and theta <= 90:
        theta += 90
    elif theta>=-180 and theta < -90:
        theta += 90
    elif theta>90 and theta <= 180:
        theta -= 270
    else:
        raise ValueError('theta out of range')
    # phi = np.rad2deg(np.arctan(z/x))+180
    phi = np.rad2deg(np.arccos(y/r))
    return [theta, phi, r, x, y, z] # [:3] sperical cood, [3:] cartesian cood

In [78]:
meta_idx = 2

print('meta_idx', meta_files[meta_idx])
with open(os.path.join(data_root, meta_files[meta_idx]), 'r') as f:
    cur_meta_data = json.load(f)

meta_idx meta_3-3.json


In [79]:
cur_meta_data['img_celeba/000085.jpg']['data_source']

'CelebA'

In [13]:
# vis = True

import tqdm

# for k in tqdm.tqdm(cur_meta_data.keys()):
#     # print(k, cur_meta_data[k]['data_source'])
#     if cur_meta_data[k]['data_source'] != data_source:
#         cur_meta_data[k]['data_source'] = data_source
#     for hk in cur_meta_data[k]['head'].keys():
#         if cur_meta_data[k]['head'][hk]['camera'] is None:
#             print('camera is None')
#             del cur_meta_data[k]['head'][hk]
#             continue
#         cur_cam = cur_meta_data[k]['head'][hk]['camera']
#         cur_TMatrix = np.array(cur_cam[:16]).reshape(4, 4)
#         cur_cam_scoord = get_cam_coords(cur_TMatrix)
#         cur_meta_data[k]['head'][hk]["camera_scoord"] = cur_cam_scoord
#         # front [45, 135]
#         # right [-45, 45]
#         # back [-135, -45]
#         # left [-180, -135], [135, 180]
#         theta = cur_cam_scoord[0]
#         if theta >= -45 and theta <= 45:
#             cur_meta_data[k]['head'][hk]['view'] = 'right'
#         elif theta >= 45 and theta <= 135:
#             cur_meta_data[k]['head'][hk]['view'] = 'front'
#         elif theta >= -135 and theta <= -45:
#             cur_meta_data[k]['head'][hk]['view'] = 'back'
#         else:
#             cur_meta_data[k]['head'][hk]['view'] = 'left'
#     if cur_meta_data[k]['head'] == {}:
#         print('head is None')
#         del cur_meta_data[k]
#         continue

In [14]:
# front_flag = False
# right_flag = False
# back_flag = False
# left_flag = False

import cv2
from utils.tool import render_camera

# def show_render_check(cur_dt, data_root):
#     cam = np.array(cur_dt['camera'])
#     r_img = render_camera(cam)
#     h, w = r_img.shape[:2]
#     a_img = cv2.imread(os.path.join(data_root, cur_dt['align_image_path']))
#     a_img = cv2.cvtColor(a_img, cv2.COLOR_BGR2RGB)
#     a_img = cv2.resize(a_img, (h, w))
#     vis_img = np.hstack([r_img, a_img])
#     display(Image.fromarray(vis_img))

# for k in tqdm.tqdm(cur_meta_data.keys()):
#     assert cur_meta_data[k]['data_source'] == data_source
#     for hk in cur_meta_data[k]['head'].keys():
#         assert cur_meta_data[k]['head'][hk]['camera'] is not None
#         cur_dt = cur_meta_data[k]['head'][hk]
#         if not front_flag:
#             if cur_dt['view'] == 'front':
#                 print(cur_dt['view'])
#                 show_render_check(cur_dt, data_root)
#                 front_flag = True
#         if not right_flag:
#             if cur_dt['view'] == 'right':
#                 print(cur_dt['view'])
#                 show_render_check(cur_dt, data_root)
#                 right_flag = True
#         if not back_flag:
#             if cur_dt['view'] == 'back':
#                 print(cur_dt['view'])
#                 show_render_check(cur_dt, data_root)
#                 back_flag = True
#         if not left_flag:
#             if cur_dt['view'] == 'left':
#                 print(cur_dt['view'])
#                 show_render_check(cur_dt, data_root)
#                 left_flag = True

In [15]:
# with open(os.path.join(data_root, meta_files[meta_idx]), 'w') as f:
#     json.dump(cur_meta_data, f, indent=4)

In [81]:
export_dir = '/data_new/chence/Exported'
os.makedirs(export_dir, exist_ok=True)

export_align_images_dir = os.path.join(export_dir, 'align_images')
export_align_parsing_dir = os.path.join(export_dir, 'align_parsing')
export_head_images_dir = os.path.join(export_dir, 'head_images')
export_head_parsing_dir = os.path.join(export_dir, 'head_parsing')
os.makedirs(export_align_images_dir, exist_ok=True)
os.makedirs(export_align_parsing_dir, exist_ok=True)
os.makedirs(export_head_images_dir, exist_ok=True)
os.makedirs(export_head_parsing_dir, exist_ok=True)

# export_start_idx = 0+7600+562+847+10168+10174+6316+10000+10000+5780+17951+10323
export_start_idx = 0+7600+562+847+10168+10174+6316
export_meta_json = os.path.join(export_dir, f'meta_{export_start_idx:07d}.json')
export_meta_dict = {}


In [82]:
export_meta_json

'/data_new/chence/Exported/meta_0029351.json'

In [83]:
for k in tqdm.tqdm(cur_meta_data.keys()):
    for hk in cur_meta_data[k]['head'].keys():
        export_folder = str(export_start_idx // 1000).zfill(5)
        export_name = 'img'+str(export_start_idx).zfill(8)+'.png'
        export_key = f'{export_folder}/{export_name}'
        export_meta_dict[export_key] = {'data_source': data_source}
        export_meta_dict[export_key].update(cur_meta_data[k]['head'][hk])
        export_start_idx += 1

100%|██████████| 6212/6212 [00:00<00:00, 277284.27it/s]


In [84]:
def work(export_key, data_item, data_root, export_align_images_dir, export_align_parsing_dir, export_head_images_dir, export_head_parsing_dir):
    try:
        export_folder = export_key.split('/')[0]
        export_align_image_folder = os.path.join(export_align_images_dir, export_folder)
        export_align_parsing_folder = os.path.join(export_align_parsing_dir, export_folder)
        export_head_image_folder = os.path.join(export_head_images_dir, export_folder)
        export_head_parsing_folder = os.path.join(export_head_parsing_dir, export_folder)
        os.makedirs(export_align_image_folder, exist_ok=True)
        os.makedirs(export_align_parsing_folder, exist_ok=True)
        os.makedirs(export_head_image_folder, exist_ok=True)
        os.makedirs(export_head_parsing_folder, exist_ok=True)

        export_align_image_path = os.path.join(export_align_images_dir, export_key)
        export_align_parsing_path = os.path.join(export_align_parsing_dir, export_key)
        export_head_image_path = os.path.join(export_head_images_dir, export_key)
        export_head_parsing_path = os.path.join(export_head_parsing_dir, export_key)

        cur_align_image_path = os.path.join(data_root, data_item['align_image_path'])
        assert os.path.exists(cur_align_image_path)
        if cur_align_image_path.endswith('.png') or cur_align_image_path.endswith('.PNG'):
            os.system(f'cp {cur_align_image_path} {export_align_image_path}')
        else:
            cur_align_image_data = cv2.imread(cur_align_image_path)
            cv2.imwrite(export_align_image_path, cur_align_image_data)

        cur_align_parsing_path = os.path.join(data_root, data_item['align_parsing_path'])
        assert os.path.exists(cur_align_parsing_path)
        assert cur_align_parsing_path.endswith('.png') or cur_align_parsing_path.endswith('.PNG')
        os.system(f'cp {cur_align_parsing_path} {export_align_parsing_path}')

        cur_head_image_path = os.path.join(data_root, data_item['head_image_path'])
        assert os.path.exists(cur_head_image_path)
        if cur_head_image_path.endswith('.png') or cur_head_image_path.endswith('.PNG'):
            os.system(f'cp {cur_head_image_path} {export_head_image_path}')
        else:
            cur_head_image_data = cv2.imread(cur_head_image_path)
            cv2.imwrite(export_head_image_path, cur_head_image_data)

        cur_head_parsing_path = os.path.join(data_root, data_item['head_parsing_path'])
        assert os.path.exists(cur_head_parsing_path)
        assert cur_head_parsing_path.endswith('.png') or cur_head_parsing_path.endswith('.PNG')
        os.system(f'cp {cur_head_parsing_path} {export_head_parsing_path}')
    except Exception as e:
        print(e)

In [85]:
import multiprocessing as mp
import cv2

global progress_bar
progress_bar = tqdm.tqdm(total=len(export_meta_dict))

def update_progress_bar(_):
    progress_bar.update()

pool = mp.Pool(processes=64)
for k, v in export_meta_dict.items():
    pool.apply_async(work, (
        k,
        v,
        data_root,
        export_align_images_dir,
        export_align_parsing_dir,
        export_head_images_dir,
        export_head_parsing_dir,
    ), callback=update_progress_bar)
pool.close()
pool.join()
progress_bar.close()

100%|██████████| 6316/6316 [03:45<00:00, 28.07it/s]


In [86]:
with open(export_meta_json, 'w') as f:
    json.dump(export_meta_dict, f, indent=4)

In [87]:
print(len(export_meta_dict))

6316


In [88]:
print('export_start_idx', export_start_idx)

export_start_idx 35667
